In [9]:
%run "./setup.ipynb"
if CLIENT == None:
    print("Failed to create CLIENT.")

Setup Complete. Use CLIENT to access Weaviate.


In [ ]:
# flush the schema and data
# CLIENT.schema.delete_all()

In [10]:
# create the schema
schema = {
    "classes": [
        {
            "class": "Work",
            "vectorizer": "text2vec-transformers",
            "moduleConfig": {
                "text2vec-transformers": {
                    "vectorizeClassName": True,
                    "resourceName": os.getenv("AZURE_OPENAI_RESOURCE_NAME"),
                    "deploymentId": os.getenv("AZURE_OPENAI_DEPLOYMENT_ID"),
                },
                "generative-openai": {
                    "vectorizeClassName": True,
                    "resourceName": os.getenv("AZURE_OPENAI_RESOURCE_NAME"),
                    "deploymentId": os.getenv("AZURE_OPENAI_DEPLOYMENT_ID"),
                }
            },
            "properties": [
                {"dataType": ["text"], "name": "title"},
                {"dataType": ["text"], "name": "accession_number"},
                {"dataType": ["text[]"], "name": "alternate_title"},
                {"dataType": ["text"], "name": "api_model"},
                {"dataType": ["text"], "name": "catalog_key"},
                {"dataType": ["text"], "name": "collection"},
                {"dataType": ["field"], "name": "collection_field"},
                {"dataType": ["text[]"], "name": "contributor"},
                {"dataType": ["text"], "name": "create_date"},
                {"dataType": ["text[]"], "name": "creator"},
                {"dataType": ["text[]"], "name": "date_created"},
                {"dataType": ["text[]"], "name": "description"},
                {"dataType": ["text[]"], "name": "genre"},
                {"dataType": ["text"], "name": "identifier"},
                {"dataType": ["text"], "name": "identifier_human_readable"},
                {"dataType": ["text"], "name": "keywords"},
                {"dataType": ["text[]"], "name": "language"},
                {"dataType": ["text"], "name": "library_unit"},
                {"dataType": ["text[]"], "name": "location"},
                {"dataType": ["text[]"], "name": "physical_description_material"},
                {"dataType": ["text[]"], "name": "physical_description_size"},
                {"dataType": ["text"], "name": "preservation_level"},
                {"dataType": ["boolean"], "name": "published"},
                {"dataType": ["text[]"], "name": "related_material"},
                {"dataType": ["text[]"], "name": "related_url"},
                {"dataType": ["text"], "name": "rights_holder"},
                {"dataType": ["text"], "name": "rights_statement"},
                {"dataType": ["text"], "name": "scope_and_contents"},
                {"dataType": ["text[]"], "name": "series"},
                {"dataType": ["text[]"], "name": "source"},
                {"dataType": ["text"], "name": "status"},
                {"dataType": ["text[]"], "name": "style_period"},
                {"dataType": ["text[]"], "name": "subject"},
                {"dataType": ["text"], "name": "table_of_contents"},
                {"dataType": ["text[]"], "name": "technique"},
                {"dataType": ["text"], "name": "visibility"},
                {"dataType": ["text"], "name": "work_type"},
            ]
        }
    ]
}

In [12]:
CLIENT.schema.delete_class("Work")
CLIENT.schema.create(schema)

UnexpectedStatusCodeException: Create class! Unexpected status code: 422, with response body: {'error': [{'message': 'vectorizer: no module with name "text2vec-openai" present'}]}.

In [ ]:
%pip install pandas
import pandas as pd

In [ ]:
# Read the csv file
data = pd.read_pickle("./data/nuldc_06_07_filtered_with_lists_(fixed_identifiers).pkl")
print(f'Number of records: {len(data)}')
first_record = data.iloc[0].to_dict()
print("Sample record: ")
for key, value in first_record.items():
    print(f'    {key}: {value}')

In [ ]:
CLIENT.batch.configure(
    batch_size=20,
    callback=weaviate.util.check_batch_result,
    dynamic=True,
    timeout_retries=5,
    connection_error_retries=5,
)

with CLIENT.batch as batch:
    for i, d in enumerate(data.iloc):
        filtered = d.dropna().to_dict()
        uuid = filtered['identifier']
        print(f"{i} / 110160 ({round(i / 110160 * 100, 1)}%) -- {uuid}")
        batch.add_data_object(data_object=filtered, class_name="Work", uuid=uuid)